In [1]:
import os
import tensorflow as tf
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix   #copy

In [2]:
data = np.load('ggt.npz')       #mark final.npz
label = data['y']
Bigdata = data['X']
Bigdata[np.isnan(Bigdata)] = 0.
#Bigdata = Bigdata[:,:,1:]
results = {}
results['acc'] = []
results['auc'] = []
results['confMat'] = []  #add
fold = 0
label[label < 5] = 0
label[label==5] = 1
y = label
index=np.arange(440)
np.random.shuffle(index)
y=y[index]
Bigdata=Bigdata[index][:,:,1:]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=22)

In [3]:
for train, test in kfold.split(np.zeros(len(y)), y):
    print("run fold:{0}".format(fold))
    train_x, train_y = Bigdata[train], y[train]
    test_x, test_y = Bigdata[test], y[test]
    #train_y, test_y = tf.one_hot(train_y, depth=6).numpy(), tf.one_hot(test_y, depth=6).numpy()

    model = tf.keras.Sequential([
    #444,500,7 -> 444 500 256
    tf.keras.layers.LSTM(256, return_sequences=True, input_shape=(500, 7)),
    #444 500 256 -> 444 500 64
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(2, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.LSTM(7, return_sequences=True),
    ])
    #2

    model.compile(optimizer='adam', loss=keras.losses.mean_squared_error)
    TensorBoard_callback = keras.callbacks.TensorBoard('logs_without_timestamp/' + str(fold) +"/" , histogram_freq=1)
    EarlyStopping_callback = keras.callbacks.EarlyStopping(patience=5)
    checkpoint_callback = keras.callbacks.ModelCheckpoint('checkpoint_without_timestamp/fold_' + str(fold) +"/" , save_best_only=True, save_weights_only=True)
    
    model.fit(x=train_x, y=train_x, batch_size=64, epochs=1, validation_split=0.2, callbacks=[TensorBoard_callback, EarlyStopping_callback, checkpoint_callback], verbose=2)


run fold:0
Train on 281 samples, validate on 71 samples
281/281 - 20s - loss: 3199018597.1246 - val_loss: 3196294551.4366
run fold:0
Train on 281 samples, validate on 71 samples
281/281 - 17s - loss: 3206928821.2954 - val_loss: 3183374945.3521
run fold:0
Train on 281 samples, validate on 71 samples
281/281 - 16s - loss: 3193202622.4057 - val_loss: 3196132878.4225
run fold:0
Train on 281 samples, validate on 71 samples
281/281 - 17s - loss: 3210685388.9822 - val_loss: 3183378129.1268
run fold:0
Train on 281 samples, validate on 71 samples
281/281 - 16s - loss: 3198628249.9644 - val_loss: 3172684028.3944
